# QA of DSL Release v1.21 - GRID Tickets only
The purpose of this notebook is to review and test the new GRID search features in 1.21

Docs: https://docs.dimensions.ai/dsl/1.21.0-preview/


## Prerequisites

Please install the latest versions of these libraries to run this notebook. 

In [84]:
import dimcli
from dimcli.shortcuts import dslquery, dslqueryall
from tqdm import tqdm
import pandas as pd
import json
dimcli.login(instance="test")
# OR EXPLICITLY: dimcli.login(user="", password="", endpoint='https://integration.ds-metrics.com')
dsl = dimcli.Dsl() 

DimCli v0.6.1 - Succesfully connected to <https://integration.ds-metrics.com> (method: dsl.ini file)


---

## DSL-247 GRID API Function `extract_affiliations`
https://uberresearch.atlassian.net/browse/DSL-247

Let's pull out some real world affiliations 

Note that sometimes empty values come through as 'None', sometimes as an empty string. So we normalize to empty string..

In [ ]:
res = dslquery("""search publications for "malaria" where authors is not empty return publications[authors]""")
testdata = []
for x in res.publications:
    for a in x['authors']:
        affiliations = a.get('affiliations')
        if affiliations:
            l = [affiliations[0].get('name', ""), affiliations[0].get('city', ""), affiliations[0].get('state', ""), affiliations[0].get('country', "")]
            l = [x if x != None else "" for x in l]
            if l not in testdata:
                testdata.append(l)
for x in enumerate(testdata[:15]): print(x)            

### Structured data

#### 1.1 single query 

In [ ]:
# implicit results 
for d in tqdm(testdata[:15]):
    res = dsl.query(f"""extract_affiliations(name="{d[0]}", city="{d[1]}", state="{d[2]}", country="{d[3]}")""")
    print(res.json)

Test also when passing results opts

In [ ]:

results_opts = ['basic', 'full', 'publisher']
for d in tqdm(testdata[:10], desc="affiliations outer loop"):
    for r in tqdm(results_opts):
        res = dsl.query(f"""extract_affiliations(name="{d[0]}", city="{d[1]}", state="{d[2]}", country="{d[3]}", results="{r}")""")
        print(res.json)

#### 1.2 batch matching

Build a JSON object from the affiliations test data above

In [ ]:
jsondata = []
for el in testdata:
    jsondata.append({"name":el[0],"city":el[1],"state":el[2],"country":el[3]})
# json.dumps(jsondata)

In [ ]:
res = dsl.query(f"""extract_affiliations(json={json.dumps(jsondata)})""")
print(res.json)

### Unstructured data

#### 2.1 Single query

In [ ]:
# implicit results 
for d in tqdm(testdata[:15]):
    res = dsl.query(f"""extract_affiliations(affiliation="{d[0]} {d[1]} {d[2]} {d[3]}")""")
    print(res.json)

Test also when passing results opts

In [ ]:
results_opts = ['basic', 'full', 'publisher']
for d in tqdm(testdata[:10], desc="affiliations outer loop"):
    for r in tqdm(results_opts):
        res = dsl.query(f"""extract_affiliations(affiliation="{d[0]} {d[1]} {d[2]} {d[3]}", results="{r}")""")
        print(res.json)

#### 2.2 Batch matching

Build a JSON object from the affiliations test data above

In [ ]:
jsondata = []
for d in testdata:
    jsondata.append({"affiliation": f"{d[0]} {d[1]} {d[2]} {d[3]}"})
# json.dumps(jsondata)

In [ ]:
res = dsl.query(f"""extract_affiliations(json={json.dumps(jsondata)})""")
print(res.json)

## DSL-207 GRID API phase 2 basic keyword and ID search 

https://uberresearch.atlassian.net/browse/DSL-207

In [37]:
%dsl search organizations where id="grid.410356.5" return organizations[basics]

Returned Organizations: 1 (total = 1)


<dimcli.Result object #4761677968. Records: 1/1>

In [39]:
%dsl search organizations where types = "Education" return organizations

Returned Organizations: 20 (total = 18775)


<dimcli.Result object #4495583056. Records: 20/18775>

In [32]:
res = dsl.query("""search organizations for "naples" return organizations limit 10""")
res.as_dataframe().head(3)

Returned Organizations: 10 (total = 10)


types   latitude state_name city_name   country_name  \
0         None        NaN    Florida    Naples  United States   
1      [Other]  26.213648    Florida    Naples  United States   
2  [Education]  40.837260       None    Naples          Italy   

                                       name  longitude  \
0                 Cancer Alliance of Naples        NaN   
1  Naples Anesthesia & Physician Associates -81.733070   
2           Parthenope University of Naples  14.253195   

                          linkout acronym             id  
0                            None    None  grid.427620.5  
1    [http://painfreenaples.com/]    NAPA  grid.477562.5  
2  [http://www.uniparthenope.it/]    None  grid.17682.3a

In [42]:
res = dsl.query("""search organizations for "naples" where state_name is not empty return organizations limit 10""")
res.as_dataframe().head(3)

Returned Organizations: 4 (total = 4)


types   latitude state_name city_name   country_name  \
0         None        NaN    Florida    Naples  United States   
1      [Other]  26.213648    Florida    Naples  United States   
2  [Nonprofit]  26.150770    Florida    Naples  United States   

                                          name  longitude  \
0                    Cancer Alliance of Naples        NaN   
1     Naples Anesthesia & Physician Associates -81.733070   
2  Naples Community Hospital Healthcare System -81.798485   

                        linkout acronym             id  
0                          None    None  grid.427620.5  
1  [http://painfreenaples.com/]    NAPA  grid.477562.5  
2       [http://www.nchmd.org/]     NCH  grid.489100.4

In [29]:
%dsldf search organizations where latitude > 5 return organizations limit 10

Returned Errors: 1
Semantic Error
Semantic errors found:
	Field 'latitude' can not be used in filters. It can only be returned in the result.


In [30]:
%dsldf search organizations where longitude > 5 return organizations limit 10

Returned Errors: 1
Semantic Error
Semantic errors found:
	Field 'longitude' can not be used in filters. It can only be returned in the result.


#### more interesting queries 

faceting works ok

In [70]:
%dsldf search organizations for "nhs" return city_name

Returned City_name: 20


id  count
0           London     38
1       Birmingham      7
2        Liverpool      7
3            Leeds      5
4       Manchester      5
5        Cambridge      3
6          Cardiff      3
7        Edinburgh      3
8           Exeter      3
9          Norwich      3
10      Nottingham      3
11       Sheffield      3
12     Southampton      3
13  Stoke-on-Trent      3
14       Wakefield      3
15         Ashford      2
16          Bolton      2
17        Bradford      2
18        Brighton      2
19         Bristol      2

searching using types

In [44]:
%dsldf search organizations where country_name="Italy" and types="Company" return organizations[basics+acronym] limit 10

Returned Organizations: 10 (total = 627)


types   latitude state_name    city_name country_name  \
0  [Company]  44.428673       None        Genoa        Italy   
1  [Company]  41.961998       None         Rome        Italy   
2  [Company]  43.780216       None     Florence        Italy   
3  [Company]  43.471596       None       Ancona        Italy   
4  [Company]  41.827152       None         Rome        Italy   
5  [Company]  41.920510       None         Rome        Italy   
6  [Company]  45.612920       None      Trieste        Italy   
7  [Company]  44.719740       None        Milan        Italy   
8  [Company]  45.467724       None        Milan        Italy   
9  [Company]  43.621750       None  Sansepolcro        Italy   

                     name  longitude  \
0         Ansaldo (Italy)   8.886877   
1  Telecom Italia (Italy)  12.461547   
2        Menarini (Italy)  11.276119   
3        Loccioni (Italy)  13.074696   
4             Eni (Italy)  12.471406   
5    Finmeccanica (Italy)  12.469245   
6      Eurospital (Italy)  13.819512   
7       UniCredit (Italy)   8.579898   
8          Edison (Italy)   9.178100   
9           Aboca (Italy)  12.120622   

                                      linkout acronym             id  
0            [http://www.ansaldoenergia.com/]    None  grid.12513.34  
1  [http://www.telecomitalia.com/tit/en.html]    None  grid.14587.3f  
2                  [http://www.menarini.com/]    None  grid.417562.3  
3          [http://www.loccioni.com/?lang=en]    None  grid.423688.3  
4        [http://www.eni.com/en_IT/home.html]    None  grid.423791.a  
5       [http://www.finmeccanica.com/en/home]    None  grid.423952.b  
6                [http://www.eurospital.com/]    None  grid.433771.5  
7    [https://www.unicreditgroup.eu/en.html#]    None  grid.436156.3  
8                     [http://www.edison.it/]    None  grid.436460.3  
9                   [http://www.aboca.com/it]    None  grid.467166.4

In [45]:
%dsldf search organizations where country_name="Italy" and types in ["Education", "Company"] return organizations[basics+acronym] limit 10

Returned Organizations: 10 (total = 854)


types   latitude      state_name city_name country_name  \
0  [Education]  45.478863            None     Milan        Italy   
1  [Education]  40.847220        Campania    Naples        Italy   
2  [Education]  46.080833            None     Udine        Italy   
3  [Education]  43.716430         Toscana      Pisa        Italy   
4  [Education]  44.402920         Liguria     Genoa        Italy   
5  [Education]  45.406387          Veneto     Padua        Italy   
6  [Education]  45.436670          Veneto    Verona        Italy   
7  [Education]  44.493890  Emilia-Romagna   Bologna        Italy   
8  [Education]  41.850150           Lazio      Rome        Italy   
9  [Education]  43.617065            None    Ancona        Italy   

                               name  longitude  \
0   Polytechnic University of Milan   9.228206   
1  University of Naples Federico II  14.256944   
2               University of Udine  13.211667   
3                University of Pisa  10.398687   
4               University of Genoa   8.958889   
5               University of Padua  11.877446   
6              University of Verona  11.003611   
7             University of Bologna  11.342778   
8    University of Rome Tor Vergata  12.597991   
9     Marche Polytechnic University  13.512690   

                                             linkout acronym           id  
0                         [http://www.polimi.it/en/]    None  grid.4643.5  
1                    [http://www.unina.it/index.jsp]    None  grid.4691.a  
2  [http://www.uniud.it/en/uniud-international?se...    None  grid.5390.f  
3                             [http://www.unipi.it/]   UniPi  grid.5395.a  
4                             [http://www.unige.it/]   UniGe  grid.5606.5  
5                 [http://www.unipd.it/en/home-page]   UNIPD  grid.5608.b  
6                [http://www.univr.it/jsp/index.jsp]    None  grid.5611.3  
7                  [http://www.unibo.it/en/homepage]   UNIBO  grid.6292.f  
8  [http://web.uniroma2.it/home.php?newlang=italian]    None  grid.6530.0  
9                      [http://www.univpm.it/Entra/]    None  grid.7010.6

In [46]:
%dsldf search organizations where id="grid.477562.5" return organizations[basics+acronym]

Returned Organizations: 1 (total = 1)


types   latitude state_name city_name   country_name  \
0  [Other]  26.213648    Florida    Naples  United States   

                                       name  longitude  \
0  Naples Anesthesia & Physician Associates  -81.73307   

                        linkout acronym             id  
0  [http://painfreenaples.com/]    NAPA  grid.477562.5

searching pubs using the new organization fields **NEAT** 

**NOTE** the following two queries are syntactically valid but won't work due to sub-querying limitations... do we have a workaround? Otherwise we should just tell people not to do that!

In [73]:
%dsldf search publications where research_orgs.country_name="Italy" and research_orgs.types="Company" 

Returned Publications: 20 (total = 9866)
WARNINGS [1]
Query is too long or complex. Please see https://docs.dimensions.ai/dsl/faq.html for more information. [code: 4]


id  year volume     type  \
0   pub.1121864022  2020     12  article   
1   pub.1120848982  2020     26  article   
2   pub.1121564312  2020     26  article   
3   pub.1121066317  2020     12  article   
4   pub.1122449006  2020    185  article   
5   pub.1113506834  2020     35  article   
6   pub.1121964341  2020    330  article   
7   pub.1121577855  2020   1726  article   
8   pub.1121884391  2020   None  chapter   
9   pub.1117495008  2019      9  article   
10  pub.1120973117  2019      5  article   
11  pub.1112739118  2019     20  article   
12  pub.1121484020  2019     78  article   
13  pub.1111369028  2019     12  article   
14  pub.1113355583  2019     20  article   
15  pub.1115227562  2019     21  article   
16  pub.1116486530  2019     10  article   
17  pub.1120396653  2019     21  article   
18  pub.1121477211  2019     10  article   
19  pub.1113179458  2019     19  article   

                                  author_affiliations      pages  \
0   [[{'first_name': 'Takafumi', 'last_name': 'Tan...        1-1   
1   [[{'first_name': 'Matteo', 'last_name': 'Buffo...        1-8   
2   [[{'first_name': 'Lorenzo', 'last_name': 'Colu...       1-10   
3   [[{'first_name': 'Takafumi', 'last_name': 'Tan...     a9-a17   
4   [[{'first_name': 'F.I.', 'last_name': 'Mulder'...      13-19   
5   [[{'first_name': 'Roberto', 'last_name': 'Rizz...    430-442   
6   [[{'first_name': 'Todd J.', 'last_name': 'Levy...     108467   
7   [[{'first_name': 'Giordano', 'last_name': 'de ...     146502   
8   [[{'first_name': 'Andrew', 'last_name': 'Livin...       1-18   
9   [[{'first_name': 'Octavio E.', 'last_name': 'S...       9220   
10  [[{'first_name': 'Andrea', 'last_name': 'Alber...         41   
11  [[{'first_name': 'Ulrich', 'last_name': 'Costa...         55   
12  [[{'first_name': 'Gloria', 'last_name': 'Lliso...       1642   
13  [[{'first_name': 'Domenico', 'last_name': 'Otr...         25   
14  [[{'first_name': 'Michael', 'last_name': 'Kreu...         71   
15  [[{'first_name': 'Ivan', 'last_name': 'Foeldva...        125   
16  [[{'first_name': 'Pratiti', 'last_name': 'Band...       2400   
17  [[{'first_name': 'Francesco', 'last_name': 'Gi...  2712-2717   
18  [[{'first_name': 'Luisa', 'last_name': 'Capalb...       4513   
19  [[{'first_name': 'Alfonso', 'last_name': 'Fasa...         50   

                                                title issue    journal.id  \
0   Autonomous network diagnosis from the carrier ...     3  jour.1138977   
1   Investigation of Current-Driven Degradation of...     2  jour.1033570   
2   Efficient and Optical Feedback Tolerant Hybrid...     2  jour.1033570   
3   Autonomous network diagnosis from the carrier ...     1  jour.1138977   
4   Edoxaban for treatment of venous thromboemboli...  None  jour.1015410   
5   An Isolated Multilevel Quasi-Resonant Multipha...     1  jour.1033560   
6   An impedance matching algorithm for common-mod...  None  jour.1089656   
7   Increases in compulsivity, inflammation, and n...  None  jour.1117575   
8   Chapter 1 Challenges and Directions for Green ...  None           NaN   
9   How typhoons trigger turbidity currents in sub...     1  jour.1045337   
10  The Role of Technology in Adherence to Physica...     1  jour.1052131   
11  Effect of pirfenidone in patients with more ad...     1  jour.1028525   
12  Synovial tissue signatures enhance clinical cl...    12  jour.1017226   
13  Effectiveness of the spot-on combination of mo...     1  jour.1039458   
14  Differing severities of acute exacerbations of...     1  jour.1028525   
15  Etanercept treatment for extended oligoarticul...     1  jour.1297499   
16  Neuronal differentiation and cell-cycle progra...     1  jour.1043282   
17  Achievement of glycaemic control is associated...    12  jour.1020801   
18  The midbody interactome reveals unexpected rol...     1  jour.1043282   
19  Characterizing advanced Parkinson’s disease: O...     1  jour.1024950   

                                        jour

In [74]:
%dsldf search publications where count(research_orgs) > 1 and research_orgs.types="Company" return research_orgs

Returned Research_orgs: 20
WARNINGS [1]
Query is too long or complex. Please see https://docs.dimensions.ai/dsl/faq.html for more information. [code: 4]


id  count              acronym      city_name    state_name  \
0   grid.410513.2  15164               Pfizer       New York      New York   
1   grid.417993.1  12410                  MSD     Kenilworth    New Jersey   
2   grid.419666.a  10993                 None          Seoul          None   
3   grid.418158.1  10870                Roche         Nutley    New Jersey   
4   grid.419481.1   9462                 None          Basel          None   
5   grid.418236.a   8962                  GSK         London          None   
6   grid.418424.f   8950                  NPC       New York      New York   
7   grid.417540.3   8837  Eli Lilly & Company   Indianapolis       Indiana   
8   grid.419815.0   7843                 None        Redmond    Washington   
9     grid.5406.7   7321                 None         Munich          None   
10  grid.418019.5   7069                  GSK   Philadelphia  Pennsylvania   
11  grid.417547.4   6622                 None          Tokyo          None   
12  grid.420044.6   6392                 None     Leverkusen          None   
13  grid.419318.6   6275                 None    Santa Clara    California   
14  grid.417886.4   6236          Amgen, Inc.  Thousand Oaks    California   
15  grid.419819.c   6052                  NTT          Tokyo          None   
16  grid.419971.3   5762                  BMS       New York      New York   
17  grid.417284.c   5681                 None      Amsterdam          None   
18  grid.168010.e   5618                   SU       Stanford    California   
19  grid.417815.e   5442                 None      Cambridge          None   

                                      name   longitude        types  \
0                   Pfizer (United States)  -73.972540    [Company]   
1                      MSD (United States)  -74.272575    [Company]   
2                    Samsung (South Korea)  127.026900    [Company]   
3                    Roche (United States)  -74.156910    [Company]   
4                   Novartis (Switzerland)    7.579728    [Company]   
5         GlaxoSmithKline (United Kingdom)   -0.316690    [Company]   
6                 Novartis (United States)  -74.502250    [Company]   
7                Eli Lilly (United States)  -86.153070    [Company]   
8                Microsoft (United States) -122.128334    [Company]   
9                        Siemens (Germany)   11.524718    [Company]   
10         GlaxoSmithKline (United States)  -75.169690    [Company]   
11                         Hitachi (Japan)  139.766080    [Company]   
12                         Bayer (Germany)    6.983162    [Company]   
13                   Intel (United States) -121.963806    [Company]   
14                   Amgen (United States) -118.838420    [Company]   
15  Nippon Telegraph and Telephone (Japan)  139.753450    [Company]   
16    Bristol-Myers Squibb (United States)  -73.972870    [Company]   
17                   Philips (Netherlands)    4.916566    [Company]   
18                     Stanford University -122.170000  [Education]   
19            AstraZeneca (United Kingdom)   -0.181646    [Company]   

      country_name   latitude  \
0    United States  40.750362   
1    United States  40.678677   
2      South Korea  37.496610   
3    United States  40.833480   
4      Switzerland  47.574320   
5   United Kingdom  51.488200   
6    United States  40.844296   
7    United States  39.756270   
8    United States  47.639720   
9          Germany  48.134320   
10   United States  40.487778   
11           Japan  35.682976   
12         Germany  51.013424   
13   United States  37.387604   
14   United States  34.173620   
15           Japan  35.689526   
16   United States  40.758110   
17     Netherlands  52.344418   
18   United States  37.430000   
19  United Kingdom  51.519590   

                                              linkout  
0                            [http://www.pfizer.com/]  
1                             [http://www.merck.com/]  
2                  [ht

## DSL-288 GRID API phase 3 all fields 

https://uberresearch.atlassian.net/browse/DSL-288

In [77]:
%dsl search organizations where id="grid.410356.5" return organizations[basics]

Returned Organizations: 1 (total = 1)


<dimcli.Result object #4779448016. Records: 1/1>

#### Getting fields dynamically

In [78]:
res = dsl.query("describe schema")

df = pd.DataFrame()

docs_for = ['organizations']
header = "sources"

d = {"sources": [], 'field': [], 'type': [], 'description':[], 'is_filter':[], 'is_entity': [],  'is_facet':[],}
for S in docs_for:
    for x in sorted(res.json[header][S]['fields']):
        d[header] += [S]
        d['field'] += [x]
        d['type'] += [res.json[header][S]['fields'][x]['type']]
        d['description'] += [res.json[header][S]['fields'][x]['description']]
        d['is_filter'] += [res.json[header][S]['fields'][x]['is_filter']]
        d['is_facet'] += [res.json[header][S]['fields'][x].get('is_facet', False)]
        d['is_entity'] += [res.json[header][S]['fields'][x].get('is_entity', False)]

fields = df.from_dict(d)
fields 

sources                     field     type  \
0   organizations                   acronym   string   
1   organizations                 city_name   string   
2   organizations                  cnrs_ids   string   
3   organizations              country_name   string   
4   organizations               established  integer   
5   organizations      external_ids_fundref   string   
6   organizations                  hesa_ids   string   
7   organizations                        id   string   
8   organizations             ificlaims_ids   string   
9   organizations                  isni_ids   string   
10  organizations                  latitude    float   
11  organizations                   linkout   string   
12  organizations                 longitude    float   
13  organizations                      name   string   
14  organizations    organization_child_ids   string   
15  organizations   organization_parent_ids   string   
16  organizations  organization_related_ids   string   
17  organizations                orgref_ids   string   
18  organizations                state_name   string   
19  organizations                     types   string   
20  organizations                  ucas_ids   string   
21  organizations                 ukprn_ids   string   
22  organizations              wikidata_ids   string   
23  organizations             wikipedia_url   string   

                                          description  is_filter  is_entity  \
0   GRID acronym of the organization. E.g., "UT" f...       True      False   
1   GRID name of the organization country. E.g., "...       True      False   
2                      CNRS IDs for this organization       True      False   
3   GRID name of the organization country. E.g., "...       True      False   
4         Year when the organization was estabilished       True      False   
5                   Fundref IDs for this organization       True      False   
6                      HESA IDs for this organization       True      False   
7   GRID ID of the organization. E.g., "grid.26999...       True      False   
8                IFI Claims IDs for this organization       True      False   
9                      ISNI IDs for this organization       True      False   
10                                               None      False      False   
11                                               None      False      False   
12                                               None      False      False   
13  GRID name of the organization. E.g., "Universi...       True      False   
14                             Child organization IDs       True      False   
15                            Parent organization IDs       True      False   
16                           Related organization IDs       True      False   
17                   OrgRef IDs for this organization       True      False   
18  GRID name of the organization country. E.g., "...       True      False   
19  Type of an organization. Available types inclu...       True      False   
20                     UCAS IDs for this organization       True      False   
21                    UKPRN IDs for this organization       True      False   
22                 WikiData IDs for this organization       True      False   
23                                      Wikipedia URL      False      False   

    is_facet  
0      False  
1       True  
2      False  
3       True  
4      False  
5      False  
6      False  
7      False  
8      False  
9      False  
10     False  
11     False  
12     False  
13     False  
14     False  
15     False  
16     False  
17     False  
18      True  
19      True  
20     False  
21     False  
22     False  
23     False

Documentation seems good 
(note: the DF needs to be built manually cause dimcli doesn't have `organizations` as a valid source yet..) 

COMMENTS

* all 'related' IDs could be called `organization_patent_ids`, `organization_child_ids`, `organization_related_ids`
* We can drop the field `other_organization_ids` for now as it's confusing 
    * TODO Eg check if 'related' is the same as in https://grid.ac/institutes/grid.410356.5 
* `external` ids: name a bit ugly and also not aligned with other external IDs we have already (eg `pmid`, `pmcid`, `altmetric_id`, `orcid_id`). Is there a chance to rationalize things here?   




#### Automated query builder 


In [59]:
%dsl search organizations where wikipedia_url is not empty return organizations

Returned Organizations: 20 (total = 97308)


<dimcli.Result object #4779353552. Records: 20/97308>

In [79]:
# one with `is not empty` for filters or attributes 
q1 = """search organizations where {} is not empty return organizations[basics+{}]"""
# one with facet 
q2 = """search organizations where {} is not empty return {}"""


for index, row in fields.iterrows():
    print("\n===\n", row['field'], "\n===")
    q = q1.format(row['field'], row['field'])
    print(">>> " + q)
    dsl.query(q)
    if row['is_facet']: 
        q = q2.format(row['field'], row['field'])
        print("\n>>> " + q)
        dsl.query(q)


===
 acronym 
===
>>> search organizations where acronym is not empty return organizations[basics+acronym]
Returned Organizations: 20 (total = 33117)

===
 city_name 
===
>>> search organizations where city_name is not empty return organizations[basics+city_name]
Returned Organizations: 20 (total = 97240)

>>> search organizations where city_name is not empty return city_name
Returned City_name: 20

===
 cnrs_ids 
===
>>> search organizations where cnrs_ids is not empty return organizations[basics+cnrs_ids]
Returned Organizations: 20 (total = 937)

===
 country_name 
===
>>> search organizations where country_name is not empty return organizations[basics+country_name]
Returned Organizations: 20 (total = 97306)

>>> search organizations where country_name is not empty return country_name
Returned Country_name: 20

===
 established 
===
>>> search organizations where established is not empty return organizations[basics+established]
Returned Organizations: 20 (total = 97308)

===
 extern

#### Data Checks 

eg against web version of https://grid.ac/institutes/grid.410356.5

In [80]:
%dsl search organizations where id="grid.410356.5" return organizations[all]

Returned Organizations: 1 (total = 1)


<dimcli.Result object #4774276880. Records: 1/1>

Let's have a look at the fields returned

In [81]:
dsl_last_results.organizations[0].keys()

dict_keys(['id', 'external_ids_fundref', 'orgref_ids', 'city_name', 'acronym', 'state_name', 'name', 'ukprn_ids', 'organization_parent_ids', 'hesa_ids', 'types', 'ucas_ids', 'country_name', 'latitude', 'ificlaims_ids', 'organization_related_ids', 'longitude', 'organization_child_ids', 'wikipedia_url', 'cnrs_ids', 'established', 'isni_ids', 'linkout', 'wikidata_ids'])

`weight` should not be there!!! 

In [82]:
dsl_last_results.organizations[0]

{'id': 'grid.410356.5',
 'external_ids_fundref': ['501100003321', '501100003322', '501100006127'],
 'orgref_ids': ['7955551'],
 'city_name': 'Kingston',
 'acronym': None,
 'state_name': 'Ontario',
 'name': "Queen's University",
 'ukprn_ids': None,
 'organization_parent_ids': None,
 'hesa_ids': None,
 'types': ['Education'],
 'ucas_ids': None,
 'country_name': 'Canada',
 'latitude': 44.225502,
 'ificlaims_ids': ['069064'],
 'organization_related_ids': ['grid.413560.5',
  'grid.413632.1',
  'grid.415354.2',
  'grid.449116.f'],
 'longitude': -76.49516,
 'organization_child_ids': None,
 'wikipedia_url': "http://en.wikipedia.org/wiki/Queen's_University",
 'cnrs_ids': None,
 'established': 1841,
 'isni_ids': ['0000 0004 1936 8331'],
 'linkout': ['http://www.queensu.ca/'],
 'wikidata_ids': ['Q1420038']}

Now let's look at the contents of the data
* external_ids_ror is missing (in the UI it's https://ror.org/02y72wh86) but we'll add that as soon as it becomes available in SOLR
